# **Fine-tuning d'un modèle de traduction EN→FR avec T5 + LoRA**

Dans le cadre du projet de LLM, nous avons développé un système de traduction automatique anglais → français basé sur les modèles modernes de type Transformer. L’objectif principal est d’adapter un modèle pré-entraîné existant afin d’obtenir une solution exploitable dans une tâche de traduction.

Nous avons choisi d’utiliser T5 (Text-to-Text Transfer Transformer) comme modèle de base, en raison de son architecture encoder–decoder particulièrement bien adaptée aux tâches de traduction. Pour affiner ses capacités sans réentraîner l’intégralité des 220 millions de paramètres, nous avons adopté la méthode LoRA (Low-Rank Adaptation), qui permet d’ajouter un petit nombre de poids entraînables tout en gelant le modèle original.

## Membres du Groupe de Projet

- Maxence KAMIONKA
- Mikhaïl BENALI
- Hadja BAH
- Emmanuel DAGNOGO


### **Import des bibliothèques nécessaires**

In [ ]:
import os
import re
import json
import random
import math
import numpy as np
import pandas as pd
from typing import Dict, List
from packaging import version
import pprint
import gc

from datasets import load_dataset, concatenate_datasets, Dataset, DatasetDict, load_from_disk
from peft import LoraConfig, TaskType, get_peft_model, PeftConfig, PeftModel

import torch
import transformers
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    TrainerCallback,
    EarlyStoppingCallback
)
import evaluate

try:
    import langid
    langid.set_languages(["en", "fr"])
    HAS_LANGID = True
except Exception:
    HAS_LANGID = False

Ici nous utilisons PyTorch afin de détécter si un GPU est disponible.

In [ ]:

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

Using device: cuda


In [ ]:
# Libérer la RAM Python
gc.collect()
# Libérer la mémoire GPU inutilisée
if torch.cuda.is_available():
    torch.cuda.empty_cache()

Nous utilisons également gc, un garbage collector, permettant de vider la mémoire RAM d'éléments inutiles.

## **Test du modèle de base**

Nous sommes partis du modèle T5-base, disponible via la bibliothèque Transformers de HuggingFace, ainsi que du tokenizer associé. Pour orienter le modèle vers la tâche de traduction, nous avons utilisé une instruction : `translate English to French`.

Cette instruction joue un rôle essentiel : dans l’architecture T5, toutes les tâches sont formulées comme une entrée textuelle. Ainsi, le modèle ne devine pas la tâche à accomplir ; il s’appuie entièrement sur le préfixe pour déterminer l’opération attendue. En ajoutant systématiquement ce préfixe au texte source, nous indiquons clairement au modèle qu’il doit réaliser une traduction de l’anglais vers le français.

In [ ]:
MODEL_NAME = "t5-base"
INSTRUCTION = "translate English to French: "  

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

C:\Users\maxka\anaconda3\envs\llm_trad\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

base_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(DEVICE)

## **Utilisation de LoRa (Low-Rank Adaption)**

On utilise LoRA afin d'entrainer des paramètres extra tout en gelant ceux du modèle de base pour ne pas lui faire oublier ce qu'il sait déjà faire originellement. On tune alors ces poids supplémentaires plutôt que tout le modèle.

Nous utilisons LoRA (Low-Rank Adaptation) afin d’entraîner uniquement un petit sous-ensemble de paramètres additionnels, tout en gelant les poids du modèle de base. Cette stratégie évite d’oublier les compétences déjà acquises par T5 lors du pré-entraînement, réduit fortement le coût mémoire/temps. 

Autrement dit, au lieu d’ajuster tous les poids de T5, nous n’entraînons que ces poids supplémentaires, ce qui permet une adaptation efficace et stable à la tâche de traduction.

In [ ]:
def load_lora_model():
    # on réutilise MODEL_NAME & tokenizer définis plus haut
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

    model.config.no_repeat_ngram_size = 3

    peft_config = LoraConfig(
        task_type=TaskType.SEQ_2_SEQ_LM,
        inference_mode=False,
        r=8,
        lora_alpha=16,
        lora_dropout=0.1,
    )

    model = get_peft_model(model, peft_config)
    model.to(DEVICE)
    model.print_trainable_parameters()

    return model

### **Choix des jeux de données**

Après des essais préliminaires, le dataset OPUS a été mis de côté : nous avons constaté des traductions un peu "étranges", susceptibles d’introduire du bruit dans le modèle.

Nous avons donc privilégié Europarl.

**Europarl en-fr** : phrases longues, style plus formel, discours parlementaires -> plus de contexte par phrase.

**On a préféré augmenter le nombre de données (ici 700K avec 0.5% soit 3500 paires en validation) plutôt qu'augmenter le nombre d'epochs (ici 2) pour éviter le sur-apprentissage et améliorer la généralisation du modèle en lui donnant davantage de phrases, tournures et vocabulaire.**

### **Charger et échantilloner le dataset OPUS et Europarl**

In [ ]:
def load_data_old(sample_size_train=80000, sample_size_val=1000):

    ### OPUS ###
    opus = load_dataset("Helsinki-NLP/opus-100", "en-fr")
    opus_train = opus["train"].shuffle(seed=42).select(range(sample_size_train))
    opus_val = opus["validation"].shuffle(seed=42).select(range(sample_size_val))

    ### EUROPARL EN-FR ###
    europarl = load_dataset("Helsinki-NLP/europarl", "en-fr", split="train").shuffle(seed=42)
    euro_train = europarl.select(range(sample_size_train))
    euro_val = europarl.select(range(sample_size_train, sample_size_train + sample_size_val))

    ### FUSION des 2 datasets ###
    train_ds = concatenate_datasets([opus_train, euro_train])
    val_ds = concatenate_datasets([opus_val, euro_val])
    
    # re-shuffle global
    train_ds = train_ds.shuffle(seed=43)
    val_ds   = val_ds.shuffle(seed=43)
    
    return train_ds, val_ds


def load_data(sample_size_train=700000, sample_size_val=3500):

    ### EUROPARL EN-FR ###
    europarl = load_dataset("Helsinki-NLP/europarl", "en-fr", split="train").shuffle(seed=42)
    euro_train = europarl.select(range(sample_size_train))
    euro_val = europarl.select(range(sample_size_train, sample_size_train + sample_size_val))
    
    return euro_train, euro_val

### **Chargement des données brutes et affichage de quelques exemples**

In [ ]:
# Charger les données brutes
train_ds, val_ds = load_data()

def show_raw_example(ds, idx=0, prefix="train"):
    ex = ds[idx]
    print(f"--- {prefix} example {idx} ---")
    print("EN :", ex["translation"]["en"])
    print("FR :", ex["translation"]["fr"])
    print()

# 3 exemples du train et 3 de la val
for i in range(3):
    show_raw_example(train_ds, i, prefix="train")

for i in range(3):
    show_raw_example(val_ds, i, prefix="val")

--- train example 0 ---
EN : As Europeans, with our experience, our culture of peace and our economic opportunities, we too are called upon to make our contribution towards a better future for Iraq.
FR : Les Européens que nous sommes, avec leur expérience, leur culture de la paix et leurs moyens économiques, sont appelés à apporter leur contribution en faveur d'un avenir meilleur en Irak.

--- train example 1 ---
EN : It does indeed speak for itself that those who are around the negotiating table are most sensitive to their own issues; this is always the case.
FR : Il est d'ailleurs évident que les personnes qui siègent autour de la table de négociations sont les plus sensibles à leurs propres problèmes. Il en est toujours ainsi.

--- train example 2 ---
EN : We nevertheless believe that the compromise is sound overall because it constitutes a clear improvement upon the original proposal.
FR : Cependant, nous estimons que le compromis est dans l’ensemble satisfaisant, en ce sens qu’il 

### **Encoder l’anglais comme input et le français comme labels**

Cette fonction prépare les données en encodant l’anglais comme entrée du modèle et le français comme cible d’apprentissage, tout en masquant les tokens de padding pour que ceux-ci n’influencent pas la fonction de perte.

In [ ]:
def preprocess_function(examples, tokenizer):
    inputs = [INSTRUCTION + ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]

    # Encodage des entrées
    model_inputs = tokenizer(
        inputs,
        max_length=256,
        truncation=True,
        padding="max_length",
    )

    # Encodage des cibles
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=256,
            truncation=True,
            padding="max_length",
        )

    label_ids = labels["input_ids"]
    pad_token_id = tokenizer.pad_token_id

    # ignorer les PAD dans la loss
    label_ids = [
        [(tok if tok != pad_token_id else -100) for tok in seq]
        for seq in label_ids
    ]

    model_inputs["labels"] = label_ids
    return model_inputs

In [ ]:
def tokenize_datasets(train, val, tokenizer):
    tokenized_train = train.map(
        lambda x: preprocess_function(x, tokenizer),
        batched=True,
        remove_columns=train.column_names,
    )
    tokenized_val = val.map(
        lambda x: preprocess_function(x, tokenizer),
        batched=True,
        remove_columns=val.column_names,
    )
    return tokenized_train, tokenized_val

### **Chargement du modèle de base et du tokenizer**

In [ ]:
# Charger modèle LoRA & réutiliser le tokenizer global
model = load_lora_model()

train_ds, val_ds = load_data()
tokenized_train, tokenized_val = tokenize_datasets(train_ds, val_ds, tokenizer)

trainable params: 884,736 || all params: 223,788,288 || trainable%: 0.3953


Map:   0%|                                                                           | 0/700000 [00:00<?, ? examples/s]C:\Users\maxka\anaconda3\envs\llm_trad\Lib\site-packages\transformers\tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|████████████████████████████████████████████████████████████████| 3500/3500 [00:00<00:00, 8235.27 examples/s]


Voici quelques exemples montrant les embeddings associés aux inputs et labels.

In [ ]:
def show_processed_example(raw_ds, tokenized_ds, idx=0, prefix="train"):
    raw = raw_ds[idx]
    tok = tokenized_ds[idx]

    print(f"--- {prefix} example {idx} ---")
    print("RAW EN :", raw["translation"]["en"])
    print("RAW FR :", raw["translation"]["fr"])
    print()

    # Inputs
    print("input_ids[:20] :", tok["input_ids"][:20])
    print("Decoded input  :", tokenizer.decode(tok["input_ids"], skip_special_tokens=True))
    print()

    # Labels (enlevant les -100 pour re-décoder)
    labels = tok["labels"]
    # On remplace les -100 par pad_token_id pour pouvoir décoder
    pad_id = tokenizer.pad_token_id
    labels_for_decode = [pad_id if x == -100 else x for x in labels]
    print("labels[:20]     :", labels[:20])
    print("Decoded labels  :", tokenizer.decode(labels_for_decode, skip_special_tokens=True))
    print()

In [ ]:
for i in range(3):
    show_processed_example(train_ds, tokenized_train, i, prefix="train")

for i in range(3):
    show_processed_example(val_ds, tokenized_val, i, prefix="val")

--- train example 0 ---
RAW EN : As Europeans, with our experience, our culture of peace and our economic opportunities, we too are called upon to make our contribution towards a better future for Iraq.
RAW FR : Les Européens que nous sommes, avec leur expérience, leur culture de la paix et leurs moyens économiques, sont appelés à apporter leur contribution en faveur d'un avenir meilleur en Irak.

input_ids[:20] : [13959, 1566, 12, 2379, 10, 282, 1611, 7, 6, 28, 69, 351, 6, 69, 1543, 13, 3065, 11, 69, 1456]
Decoded input  : translate English to French: As Europeans, with our experience, our culture of peace and our economic opportunities, we too are called upon to make our contribution towards a better future for Iraq.

labels[:20]     : [622, 2430, 3890, 35, 7, 238, 678, 7056, 6, 393, 1089, 11183, 6, 1089, 1543, 20, 50, 25060, 3, 15]
Decoded labels  : Les Européens que nous sommes, avec leur expérience, leur culture de la paix et leurs moyens économiques, sont appelés à apporter leur 

### **Métriques d'évaluation de la traduction**

Nous avons utilisé 3 métriques afin de mesurer les performances du modèle : 

La métrique BLEU permet de comparer des n-grammes entre la prédiction et le label afin de mesurer la proximité de la réponse générée avec les différentes réponses attendues.

Le ROUGE est principalement utilisé pour évaluer la qualité des résumés automatiques. Il mesure le recall des n-grammes entre le résumé généré et un ou plusieurs résumés de référence. ROUGE-1 évalue les mots individuels, tandis que ROUGE-2 évalue les paires de mots. Un score ROUGE élevé indique que le résumé couvre bien les informations clés du texte original.

Le METEOR est une métrique plus flexible que BLEU, car elle prend en compte non seulement la précision des mots, mais aussi les synonymes et leur ordre. Elle utilise des techniques de recall et de précision, ainsi qu’un alignement entre les mots du texte généré et ceux de référence. METEOR est souvent utilisé pour évaluer les traductions et les résumés, parce qu'il offre une évaluation plus nuancée de la qualité linguistique.

In [ ]:
sacrebleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

## Pour nettoyer le texte
def postprocess_text(preds, labels):
    preds = [p.strip() for p in preds]
    labels = [l.strip() for l in labels]
    return preds, labels


def compute_metrics(eval_preds, tokenizer):
    # eval_preds peut être un tuple (preds, labels)
    # ou un objet EvalPrediction avec .predictions et .label_ids
    if hasattr(eval_preds, "predictions"):
        preds = eval_preds.predictions
        labels = eval_preds.label_ids
    else:
        preds, labels = eval_preds

    # Certains modèles renvoient (logits, ...) -> on garde seulement le 1er élément
    if isinstance(preds, tuple):
        preds = preds[0]

    # On met tout en np.array pour être tranquille
    preds = np.array(preds)

    # Cas où preds = logits (batch, seq_len, vocab_size) -> on prend l'argmax
    if preds.ndim == 3:
        preds = np.argmax(preds, axis=-1)

    # On s'assure que ce sont bien des entiers
    preds = preds.astype("int64")

    # si jamais il y a des valeurs négatives dans preds on les remplace par pad_token_id avant decode
    preds[preds < 0] = tokenizer.pad_token_id

    # Gestion des labels : on remet pad_token_id à la place des -100 pour décoder
    labels = np.array(labels)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Decode des prédictions et des labels
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Nettoyage simple
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # sacreBLEU / METEOR / ROUGE
    # sacreBLEU attend une liste de listes pour les références
    refs_list = [[r] for r in decoded_labels]

    bleu_res = sacrebleu.compute(
        predictions=decoded_preds,
        references=refs_list,
    )

    rouge_res = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
    )

    meteor_res = meteor.compute(
        predictions=decoded_preds,
        references=decoded_labels,
    )

    return {
        "bleu": bleu_res["score"],
        "meteor": meteor_res["meteor"],
        "rouge1": rouge_res["rouge1"],
        "rouge2": rouge_res["rouge2"],
        "rougeL": rouge_res["rougeL"],
    }

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maxka\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\maxka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\maxka\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## **Métriques sur modèle de base**

Nous avons d’abord évalué les performances du modèle de base, afin de disposer d’un point de comparaison et de mesurer ensuite l’impact du fine-tuning sur les métriques obtenues.

In [ ]:
# Modèle de base (non fine-tuné)
model_base = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(DEVICE)

model_base.config.no_repeat_ngram_size = 3

data_collator_base = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model_base,
)

eval_args_base = Seq2SeqTrainingArguments(
    output_dir="baseline_t5_base_en_fr",
    per_device_eval_batch_size=8,
    gradient_accumulation_steps = 2, 
    predict_with_generate=True,
    generation_max_length=128,
    generation_num_beams=4,
    do_train=False,
    do_eval=True,
    logging_dir="logs_baseline",
)

trainer_base = Seq2SeqTrainer(
    model=model_base,
    args=eval_args_base,
    eval_dataset=tokenized_val,  # même val que pour LoRA
    tokenizer=tokenizer,
    data_collator=data_collator_base,
    compute_metrics=lambda p: compute_metrics(p, tokenizer),
)

baseline_metrics = trainer_base.evaluate()
print("Baseline metrics :")
pprint.pprint(baseline_metrics)

C:\Users\maxka\AppData\Local\Temp\ipykernel_38968\32419789.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer_base = Seq2SeqTrainer(
C:\Users\maxka\anaconda3\envs\llm_trad\Lib\site-packages\transformers\generation\utils.py:1733: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Baseline metrics :
{'eval_bleu': 37.108577481672434,
 'eval_loss': 0.9090045094490051,
 'eval_meteor': 0.6088466325807146,
 'eval_model_preparation_time': 0.003,
 'eval_rouge1': 0.6654090098650332,
 'eval_rouge2': 0.4747695313409016,
 'eval_rougeL': 0.6229810019071551,
 'eval_runtime': 749.3046,
 'eval_samples_per_second': 4.671,
 'eval_steps_per_second': 0.585}


Ici on crée un callback qui va être appelé à la fin de chaque epoch et qui permettra de vider la mémoire inutilisée.

In [ ]:
class GarbageCollectorCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        # Libérer la RAM Python
        gc.collect()
        # Libérer la mémoire GPU inutilisée
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        return control

## **Trainer**

On crée ici une fonction d'entrainement.
Cette fonction encapsule l’ensemble du processus d’entraînement du modèle. Elle définit les paramètres du fine-tuning (batch size, learning rate, accumulation de gradients, nombre d’époques, stratégies de sauvegarde et d’évaluation), initialise un Seq2SeqTrainer(entraîneur optimisé pour les tâches où une séquence doit être transformée en une autre) et lance l’apprentissage tout en calculant les métriques à chaque époque. 

À l’issue du processus, le meilleur modèle ainsi que le tokenizer sont sauvegardés pour une utilisation ultérieure.

In [ ]:

# per_device_eval_batch_size=8 
# gradient_accumulation_steps = 2 réduire la mémoire : stabilité d’un batch effectif de 16, mais avec la mémoire d’un batch de 8 seulement, mais 2 fois plus long

def train(model, tokenizer, tokenized_train, tokenized_val):
    training_args = Seq2SeqTrainingArguments(
        output_dir="finetuned_t5_base_en_fr",
        learning_rate=3e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        gradient_accumulation_steps = 2, 
        num_train_epochs=2,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="epoch",
        predict_with_generate=True,
        load_best_model_at_end=True,
        metric_for_best_model="eval_bleu",
        greater_is_better=True,
        logging_steps=50,

        generation_max_length=128,
        generation_num_beams=4,
    )

    gc_callback = GarbageCollectorCallback()

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        tokenizer=tokenizer,
        compute_metrics=lambda p: compute_metrics(p, tokenizer),
        callbacks=[gc_callback],
    )

    trainer.train()

    trainer.model.save_pretrained(training_args.output_dir)
    tokenizer.save_pretrained(training_args.output_dir)
    print("Best eval :", trainer.state.best_metric)
    print("Best checkpoint :", trainer.state.best_model_checkpoint)

    return trainer

## **Entrainement du modèle**

On entraine notre modèle et on l'évalue avec notre jeu de validation

In [18]:
# 1) Entraîner le modèle LoRA
trainer = train(model, tokenizer, tokenized_train, tokenized_val)

# 2) Évaluer proprement le modèle LoRA sur le même jeu de validation
lora_metrics = trainer.evaluate(eval_dataset=tokenized_val)
print("Métriques modèle de base :")
pprint.pprint(baseline_metrics)
print("Métriques modèle LoRA :")
pprint.pprint(lora_metrics)

C:\Users\maxka\AppData\Local\Temp\ipykernel_38968\2911972018.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



On montre l'évolution des métriques au fur et à mesure de l'entrainement

In [ ]:

logs = pd.DataFrame(trainer.state.log_history)

# Lignes d'éval (celles qui ont une eval_loss)
eval_logs = logs[logs["eval_loss"].notna()]

# Colonnes qui nous intéressent
cols = ["epoch", "step", "eval_loss", "eval_bleu", "eval_meteor", "eval_rougeL"]

print(eval_logs[cols])

      epoch   step  eval_loss  eval_bleu  eval_meteor  eval_rougeL
875     1.0  43750   0.684410  40.336185     0.629312      0.64225
1751    2.0  87500   0.683656  40.330786     0.629337      0.64239
1753    2.0  87500   0.684410  40.336185     0.629312      0.64225


Le modèle T5-base avait obtenu un BLEU de 37.10, un METEOR de 0.608 et un ROUGE-L de 0.622. Après fine-tuning, ces scores montent respectivement à environ 40.33, 0.629 et 0.643. On observe donc une amélioration cohérente sur toutes les métriques, avec un gain  de plus de 3 points BLEU, indiquant des traductions plus fidèles et mieux alignées. La légère hausse des scores METEOR et ROUGE confirme une meilleure couverture lexicale et une structure de phrase plus proche des références.

## **Recharger le modèle fine-tuné**

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

BASE_MODEL_NAME = "t5-base"
PEFT_DIR = "./finetuned_t5_base_en_fr"


Using device: cuda


## Charger le modèle de base et LoRA 
Ici, nous chargeons le modèle de base , le tokenizer ainsi que le modèle LoRA fine-tuné à des fins de tests et de comparaison.

In [21]:
# 1) Tokenizer commun (T5-base) : on le charge "UNE" seule fois (sauf si vous faites un run de tout) :)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)

# 2) Modèle de base (non fine-tuné)
base_model = AutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL_NAME).to(DEVICE)
base_model.eval()

# 3) Modèle LoRA fine-tuné (chargé depuis le dossier PEFT_DIR)
lora_backbone = AutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL_NAME)
lora_model = PeftModel.from_pretrained(
    lora_backbone,
    PEFT_DIR,
    local_files_only=True,
).to(DEVICE)
lora_model.eval()

print("Modèle de base + modèle LoRA rechargés depuis les chemins.")


Modèle de base + modèle LoRA rechargés depuis les chemins.


## **Fonction de traduction + tests**

In [ ]:
def translate_sentence(sentence, model, tokenizer, max_length=256, num_beams=4):
    model.eval()

    inputs = tokenizer(
        INSTRUCTION + sentence,
        return_tensors="pt",
        truncation=True,
        max_length=max_length,
    ).to(DEVICE)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            num_beams=num_beams,
            no_repeat_ngram_size=3,  # comme dans model_base.config
        )
        
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

### **Tests avec quelques phrases du dataset train**
Nous avons pris quelques exemples du dataset d’entraînement et les avons passés dans le modèle de base ainsi que dans le modèle fine-tuné. Cela permet simplement de vérifier que les modèles traduisent correctement les phrases, sans forcément reproduire exactement les références du train. 
Les deux modèles produisent des traductions proches du sens attendu, et une petite différence notable apparaît : le modèle fine-tuné traduit Irak par Iraq, alors que le modèle de base conserve la forme anglaise.

In [23]:
# Quelques phrases du dataset train
for i in range(3):
    en = train_ds[i]["translation"]["en"]
    fr_gold = train_ds[i]["translation"]["fr"]

    fr_base = translate_sentence(en, base_model, tokenizer)
    fr_lora = translate_sentence(en, lora_model, tokenizer) # modèle fine-tuné LoRA

    print("--- Exemple train", i, "---")
    print("EN      :", en)
    print("FR gold :", fr_gold)
    print("FR base :", fr_base)
    print("FR LoRA :", fr_lora)
    print()

--- Exemple train 0 ---
EN      : As Europeans, with our experience, our culture of peace and our economic opportunities, we too are called upon to make our contribution towards a better future for Iraq.
FR gold : Les Européens que nous sommes, avec leur expérience, leur culture de la paix et leurs moyens économiques, sont appelés à apporter leur contribution en faveur d'un avenir meilleur en Irak.
FR base : En tant qu'Européens, avec notre expérience, notre culture de paix et nos possibilités économiques, nous sommes également appelés à contribuer à un avenir meilleur pour l'Iraq.
FR LoRA : En tant qu'Européens, avec notre expérience, notre culture de paix et nos perspectives économiques, nous sommes également appelés à contribuer à un avenir meilleur pour l'Irak.

--- Exemple train 1 ---
EN      : It does indeed speak for itself that those who are around the negotiating table are most sensitive to their own issues; this is always the case.
FR gold : Il est d'ailleurs évident que les 

### **Tests avec quelques phrases du dataset validation**

In [24]:
# Quelques phrases du dataset validation
for i in range(3):
    en = val_ds[i]["translation"]["en"]
    fr_gold = val_ds[i]["translation"]["fr"]

    fr_base = translate_sentence(en, base_model, tokenizer)
    fr_lora = translate_sentence(en, lora_model, tokenizer)  # modèle fine-tuné LoRA

    print("--- Exemple val", i, "---")
    print("EN      :", en)
    print("FR gold :", fr_gold)
    print("FR base :", fr_base)
    print("FR LoRA :", fr_lora)
    print()

--- Exemple val 0 ---
EN      : Do you accept, Commissioner, that what we are facing in our single market is a double distortion, a distortion between beer and wine, which is grossly unfair, and the distortion arises that because of grossly different rates of taxation and excise duty between Member States.
FR gold : (EN) Le commissaire admet-il que dans notre marché unique, nous sommes confrontés à une double inégalité, à savoir une inégalité entre la bière et le vin, ce qui est extrêmement injuste, et une inégalité imputable à la grande différence entre les taux d'imposition et les droits d'accises d'un État membre à l'autre.
FR base : Acceptez-vous, Monsieur le Commissaire, que ce que nous faisons dans notre marché unique est une double distorsions, une distorsion entre la bière et le vin, qui est grossièrement injuste, en raison de taux d'imposition ou d’accises nettement différents entre les États membres?
FR LoRA : Acceptez-vous, Monsieur le Commissaire, que ce que nous avons à fa

### **Tests avec quelques phrases de test personnelles**

In [25]:
# phrases de test perso
tests = [
    "Hello, how are you?",
    "This project is about automatic translation.",
    "The weather is nice today.",
]

for s in tests:
    print("EN      :", s)
    print("FR base :", translate_sentence(s, base_model, tokenizer))
    print("FR LoRA :", translate_sentence(s, lora_model, tokenizer))
    print()


EN      : Hello, how are you?
FR base : Bonjour, comment êtes-vous?
FR LoRA : Bonjour, comment êtes-vous?

EN      : This project is about automatic translation.
FR base : Ce projet concerne la traduction automatique.
FR LoRA : Ce projet concerne la traduction automatique.

EN      : The weather is nice today.
FR base : Le temps est agréable aujourd'hui.
FR LoRA : Le temps est agréable aujourd'hui.



# **Traduction en sens inverse : FR --> EN**

Après avoir fine-tuné T5 pour la traduction EN→FR, nous avons testé l’idée de simplement changer l’instruction pour obtenir l’inverse (FR→EN). Cependant, nous avons rapidement constaté que cela ne fonctionne pas : le modèle avait été entraîné uniquement dans un sens, avec des couples entrée→sortie bien définis, et il renvoyait donc systématiquement le même texte en sortie lorsqu’on lui demandait l’autre direction.

Certaines architectures permettent effectivement la traduction bidirectionnelle, mais cela nécessite un entraînement dans les deux sens. Faute de temps et de ressources, nous avons donc entraîné un second modèle dédié au FR→EN, ce qui nous a permis au passage de mieux comprendre le comportement de T5 et ses limites.

### **Choix des datasets**
Pour entraîner le modèle FR→EN, nous sommes partis du constat que le modèle fine-tuné uniquement sur Europarl produisait de bonnes traductions pour les phrases longues et formelles, mais qu’il avait du mal avec les phrases simples ou plus informelles. 

En effet, Europarl correspond à un registre soutenu (discours parlementaires) et manque de variété.
Pour corriger cela, nous avons donc mélangé trois datasets complémentaires :

- Europarl (phrases longues, vocabulaire formel),

- Opus Books (phrases littéraires, style varié),

- Francophonia (phrases courtes, quotidiennes, beaucoup plus simples).

Ce mix permet d’équilibrer le registre linguistique et de créer un dataset plus représentatif pour la traduction générale.

### **Phase préparatoire des données**

Le script applique un pipeline complet :

- Nettoyage des phrases (URLs, balises, caractères non imprimables, répétitions inutiles, etc.).

- Filtrage (phrases identiques, code ou timestamps, mismatch de langue FR/EN).

- Déduplication pour éviter d’entraîner plusieurs fois sur les mêmes couples.

- Échantillonnage équilibré (123k Opus, 300k Europarl, 318k Francophonia).

- Fusion et mélange des datasets.

- Split train / validation / test (97% / 1.5% / 1.5%).

- Sauvegarde des données nettoyées 

In [ ]:
RNG_SEED = 42
random.seed(RNG_SEED)

OUT_DIR = "data_mt_fr_en"
os.makedirs(OUT_DIR, exist_ok=True)

# 1) Chargement des datasets (remappés FR->EN)

def _ensure_dataset(ds_or_dd: Dataset | DatasetDict) -> Dataset:
    """Convertit un DatasetDict en Dataset unique (concat de tous les splits)."""
    if isinstance(ds_or_dd, DatasetDict):
        parts = [v for k, v in ds_or_dd.items()]
        if len(parts) == 1:
            return parts[0]
        return concatenate_datasets(parts)
    return ds_or_dd

def load_opus_books_fr_en() -> Dataset:
    """
    opus_books config 'en-fr' -> on remappe FR->EN:
      src_text = FR  (pair['fr'])
      tgt_text = EN  (pair['en'])
    """
    ds = load_dataset("opus_books", "en-fr")  # souvent DatasetDict
    ds = _ensure_dataset(ds)
    ds = ds.rename_column("translation", "pair")
    ds = ds.map(lambda x: {
        "src_text": x["pair"].get("fr", ""),   # FR
        "tgt_text": x["pair"].get("en", ""),   # EN
        "source": "opus_books"
    }, remove_columns=["pair"])
    return ds

def load_europarl_fr_en() -> Dataset:
    """
    Europarl config 'en-fr' -> remap FR->EN
    """
    ds = load_dataset("Helsinki-NLP/europarl", "en-fr", split="train")
    ds = ds.rename_column("translation", "pair")
    ds = ds.map(lambda x: {
        "src_text": x["pair"].get("fr", ""),   # FR
        "tgt_text": x["pair"].get("en", ""),   # EN
        "source": "europarl"
    }, remove_columns=["pair"])
    return ds

def load_francophonia_fr_en() -> Dataset:
    """
    FrancophonIA/english_french -> colonnes 'english', 'french'
    On remappe FR->EN.
    """
    ds = load_dataset("FrancophonIA/english_french", split="train")
    ds = ds.rename_column("french", "src_text")   # FR
    ds = ds.rename_column("english", "tgt_text")  # EN
    ds = ds.map(lambda x: {"source": "francophonia"})
    return ds

# 2) Pipeline de nettoyage

_ws_re = re.compile(r"\s+")
_ctrl_re = re.compile(r"[\u0000-\u0008\u000B-\u000C\u000E-\u001F]")
_url_re = re.compile(r"https?://\S+|www\.\S+")
_tag_re = re.compile(r"<[^>]+>")
_repeated_punct_re = re.compile(r"([!?.,;:\-–—])\1{2,}")
_non_printable_re = re.compile(r"[^\x09\x0A\x0D\x20-\x7E\u00A0-\uD7FF\uE000-\uFFFD]")

def normalize(text: str) -> str:
    text = text.replace("\u00A0", " ")
    text = _ctrl_re.sub(" ", text)
    text = _non_printable_re.sub(" ", text)
    text = _tag_re.sub(" ", text)
    text = _url_re.sub(" ", text)
    text = _repeated_punct_re.sub(r"\1", text)
    text = _ws_re.sub(" ", text).strip()
    return text

def ascii_ratio(s: str) -> float:
    if not s:
        return 1.0
    ascii_chars = sum(1 for ch in s if ord(ch) < 128)
    return ascii_chars / max(1, len(s))

def punctuation_ratio(s: str) -> float:
    if not s:
        return 0.0
    punct = sum(1 for ch in s if ch in ".,;:!?…\"'()[]{}<>/\\|@#$%^&*_+=~`")
    return punct / max(1, len(s))

def looks_code_like(s: str) -> bool:
    patterns = [
        r"^\d{1,2}:\d{2}(:\d{2})?$",    # 01:23:45
        r"^\[\w+\]$",                   # [MUSIC], [LAUGH]
        r"\b(function|var|let|const|<script|</script>)\b",
        r"{\s*}$", r";\s*$"
    ]
    return any(re.search(p, s) for p in patterns)

def length_ok(src: str, tgt: str,
              min_len=1, max_len=200,
              max_len_ratio=3.0) -> bool:
    ls, lt = len(src.split()), len(tgt.split())
    if ls < min_len or lt < min_len:
        return False
    if ls > max_len or lt > max_len:
        return False
    ratio = max(ls, lt) / max(1, min(ls, lt))
    return ratio <= max_len_ratio

def lang_ok(src: str, tgt: str) -> bool:
    """FR -> EN attendu."""
    if not HAS_LANGID:
        return True
    s_lang, s_conf = langid.classify(src)  # should be fr
    t_lang, t_conf = langid.classify(tgt)  # should be en
    return (s_lang == "fr" and t_lang == "en" and s_conf > 0.85 and t_conf > 0.85)

def clean_example(ex: Dict) -> Dict:
    s = normalize(ex["src_text"])
    t = normalize(ex["tgt_text"])
    return {"src_text": s, "tgt_text": t, "source": ex["source"]}

def filter_example(ex: Dict) -> bool:
    s = ex["src_text"]; t = ex["tgt_text"]
    if not s or not t:
        return False
    if s == t:
        return False
    if looks_code_like(s) or looks_code_like(t):
        return False
    if punctuation_ratio(s) > 0.35 or punctuation_ratio(t) > 0.35:
        return False
    # Pour FR->EN: on vérifie l'ASCII sur la cible anglaise (t)
    if ascii_ratio(t) < 0.75:
        return False
    if not length_ok(s, t):
        return False
    if not lang_ok(s, t):
        return False
    return True

def dedupe(ds: Dataset) -> Dataset:
    """
    Déduplication (src_text, tgt_text), compatible anciennes versions.
    """
    if hasattr(ds, "drop_duplicates"):
        try:
            return ds.drop_duplicates(subset=["src_text", "tgt_text"])
        except TypeError:
            return ds.drop_duplicates(column_names=["src_text", "tgt_text"])
    
    df = ds.to_pandas()
    df = df.drop_duplicates(subset=["src_text", "tgt_text"]).reset_index(drop=True)
    return Dataset.from_pandas(df, preserve_index=False)


# 3) Chargement + nettoyage

print("Chargement des datasets...")
opus_books = load_opus_books_fr_en()
europarl   = load_europarl_fr_en()
franco     = load_francophonia_fr_en()

for name, ds in [("opus_books", opus_books), ("europarl", europarl), ("francophonia", franco)]:
    print(f"- {name}: {len(ds):,} paires avant nettoyage")

def apply_cleaning(ds: Dataset, name: str) -> Dataset:
    ds = ds.map(clean_example, desc=f"normalize::{name}")
    ds = ds.filter(filter_example, desc=f"filter::{name}")
    ds = dedupe(ds)
    print(f"  -> {name}: {len(ds):,} paires après nettoyage")
    return ds

opus_books = apply_cleaning(opus_books, "opus_books")
europarl   = apply_cleaning(europarl,   "europarl")
franco     = apply_cleaning(franco,     "francophonia")


# 4) Échantillonnage & mix

TARGET_MAX = {
    "opus_books": 300_000,
    "europarl":   300_000,
    "francophonia": 400_000
}

def cap(ds: Dataset, cap_size: int, seed=RNG_SEED) -> Dataset:
    if len(ds) <= cap_size:
        return ds
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    return ds.select(idx[:cap_size])

opus_cap = cap(opus_books, TARGET_MAX["opus_books"])
euro_cap = cap(europarl,   TARGET_MAX["europarl"])
fran_cap = cap(franco,     TARGET_MAX["francophonia"])

print("Tailles après cap:")
print(f"  opus_books: {len(opus_cap):,}")
print(f"  europarl:   {len(euro_cap):,}")
print(f"  francophonia:{len(fran_cap):,}")

# Mix (pondération légère possible)
mix_parts = [opus_cap, fran_cap, euro_cap]
mix = concatenate_datasets(mix_parts).shuffle(seed=RNG_SEED)
print("Taille totale après mix:", len(mix))


# 5) Split train/validation/test

def split_dataset(ds: Dataset,
                  train_ratio=0.97,
                  valid_ratio=0.015,
                  seed=RNG_SEED) -> DatasetDict:
    n = len(ds)
    idx = list(range(n))
    random.Random(seed).shuffle(idx)
    n_train = int(n * train_ratio)
    n_valid = int(n * valid_ratio)
    train_idx = idx[:n_train]
    valid_idx = idx[n_train:n_train + n_valid]
    test_idx  = idx[n_train + n_valid:]
    return DatasetDict({
        "train": ds.select(train_idx),
        "validation": ds.select(valid_idx),
        "test": ds.select(test_idx)
    })

dsdict = split_dataset(mix)
for split in dsdict:
    print(split, len(dsdict[split]))

# 6) Sauvegarde (Arrow/Parquet + JSONL)

ARROW_DIR   = os.path.join(OUT_DIR, "arrow")
PARQUET_DIR = os.path.join(OUT_DIR, "parquet")
JSONL_DIR   = os.path.join(OUT_DIR, "jsonl")
os.makedirs(ARROW_DIR, exist_ok=True)
os.makedirs(PARQUET_DIR, exist_ok=True)
os.makedirs(JSONL_DIR, exist_ok=True)

dsdict.save_to_disk(ARROW_DIR)

for split, ds in dsdict.items():
    ds.to_parquet(os.path.join(PARQUET_DIR, f"{split}.parquet"))

def export_jsonl(ds: Dataset, path: str):
    with open(path, "w", encoding="utf-8") as f:
        for ex in ds:
            f.write(json.dumps(
                {"src": ex["src_text"], "tgt": ex["tgt_text"], "source": ex["source"]},
                ensure_ascii=False
            ) + "\n")

for split, ds in dsdict.items():
    export_jsonl(ds, os.path.join(JSONL_DIR, f"{split}.jsonl"))

def sample_preview(ds: Dataset, k=5):
    k = min(k, len(ds))
    ids = random.sample(range(len(ds)), k) if k > 0 else []
    return [ds[i] for i in ids]

print("\nAperçu échantillon (validation) FR→EN:")
for ex in sample_preview(dsdict["validation"], 5):
    print(f"[{ex['source']}] FR: {ex['src_text']}  ||  EN: {ex['tgt_text']}")

print("\nTerminé. Données prêtes dans:", OUT_DIR)

Chargement des datasets...


README.md: 0.00B [00:00, ?B/s]

en-fr/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

Map:   0%|          | 0/127085 [00:00<?, ? examples/s]

...

Saving the dataset (0/1 shards):   0%|          | 0/11144 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]


Aperçu échantillon (validation) FR→EN:
[europarl] FR: Nous avons d’ailleurs proposé des amendements qui rompent avec le principe du pays d’origine et excluent les services économiques d’intérêt général de son champ d’application.  ||  EN: We have, moreover, tabled some amendments that break with the country of origin principle and that exclude services of general economic interest from the scope of the directive.
[francophonia] FR: Dans la vie de tous les jours nous avons beaucoup d'obligations et de responsabilités.  ||  EN: In everyday life we have many obligations and responsibilities.
[europarl] FR: Pour pouvoir faire une véritable évaluation et obtenir des statistiques fiables, nous devons cependant définir une stratégie afin de déterminer quelles informations les États membres doivent communiquer à l'Observatoire.  ||  EN: In order to be able to carry out a credible assessment and produce reliable statistics, we must however have a strategy concerning what information the Member

### **Entrainement et Evaluation du modèle**
Ici, on entraîne le modèle puis on l'évalue sur des données démo.
On affiche sa loss tout au long de l'entraînement.

In [ ]:
# Entraînement T5-small FR→EN
DATA_DIR = "data_mt_fr_en/arrow"     
MODEL_NAME = "t5-small"
OUT_DIR = "outputs/t5-fren"

SEED = 42
MAX_SOURCE_LEN = 128
MAX_TARGET_LEN = 128
BATCH_SIZE = 16
GRAD_ACCUM = 2
LR = 3e-4
NUM_EPOCHS = 3
WARMUP_RATIO = 0.06
LABEL_SMOOTHING = 0.1
BEAM_SIZE = 4
GEN_MAX_NEW_TOKENS = 128

os.makedirs(OUT_DIR, exist_ok=True)
random.seed(SEED)
torch.manual_seed(SEED)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n[TRAIN] Device: {device}")

print("[TRAIN] Chargement du DatasetDict...")
dsdict = load_from_disk(DATA_DIR)
print({k: len(v) for k, v in dsdict.items()})


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)

PREFIX = "translate French to English: "

def preprocess_function(batch):
    # FR->EN : input = FR (src_text), target = EN (tgt_text)
    inputs = [PREFIX + s for s in batch["src_text"]]
    model_inputs = tokenizer(inputs, max_length=MAX_SOURCE_LEN, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch["tgt_text"], max_length=MAX_TARGET_LEN, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

print("[TRAIN] Tokenisation...")
tokenized = {}
for split in ["train", "validation", "test"]:
    if split in dsdict:
        tokenized[split] = dsdict[split].map(
            preprocess_function,
            batched=True,
            remove_columns=dsdict[split].column_names,
            desc=f"tokenize::{split}"
        )

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

metric_bleu = evaluate.load("sacrebleu")
metric_chrf = evaluate.load("chrf")

def postprocess_text(preds: List[str], labels: List[str]):
    preds = [p.strip() for p in preds]
    labels = [l.strip() for l in labels]
    return preds, [[l] for l in labels]

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = [[(l if l != -100 else tokenizer.pad_token_id) for l in label] for label in labels]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    bleu = metric_bleu.compute(predictions=decoded_preds, references=decoded_labels)
    chrf = metric_chrf.compute(predictions=decoded_preds, references=[l[0] for l in decoded_labels])
    gen_lens = [ (pred != "" and len(tokenizer(pred).input_ids)) or 0 for pred in decoded_preds ]
    return {"bleu": round(bleu["score"], 4), "chrf": round(chrf["score"], 4),
            "gen_len": round(sum(gen_lens) / max(1, len(gen_lens)), 2)}

# 6) Entraînement (compat toutes versions)

TRANSF_VER = version.parse(transformers.__version__)
print("[TRAIN] Transformers version:", TRANSF_VER)
LEGACY = False

common_kwargs = dict(
    output_dir=OUT_DIR,
    overwrite_output_dir=True,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=LR,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    seed=SEED,
    logging_steps=200,
    save_steps=1000,
    save_total_limit=2,
)

try:
    training_args = Seq2SeqTrainingArguments(
        **common_kwargs,
        evaluation_strategy="steps",
        save_strategy="steps",
        logging_strategy="steps",
        warmup_ratio=WARMUP_RATIO,
        label_smoothing_factor=LABEL_SMOOTHING,
        predict_with_generate=True,
        generation_max_length=GEN_MAX_NEW_TOKENS,
        generation_num_beams=BEAM_SIZE,
        fp16=torch.cuda.is_available(),
        bf16=False,
        dataloader_num_workers=2,
        report_to="none",
    )
except TypeError:
    LEGACY = True
    print("⚠️  Mode LEGACY activé : évaluation intégrée indisponible, on évaluera après l'entraînement.")
    training_args = Seq2SeqTrainingArguments(**common_kwargs)

callbacks = []
try:
    
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
except Exception:
    callbacks = []

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=None if LEGACY else tokenized.get("validation"),
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=None if LEGACY else compute_metrics,
    callbacks=callbacks if not LEGACY else None
)

print("[TRAIN] Démarrage de l'entraînement...")
trainer.train()

# Sauvegardes
trainer.save_model(OUT_DIR)
tokenizer.save_pretrained(OUT_DIR)

# 7) Évaluation
def generate_translate(batch_texts, max_new_tokens=GEN_MAX_NEW_TOKENS, num_beams=BEAM_SIZE):
    inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, num_beams=num_beams)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

def eval_split_manual(split_ds):
    preds, refs = [], []
    B = 64
    buf_src = []
    for ex in split_ds:
        buf_src.append(PREFIX + ex["src_text"])  # FR input (avec préfixe)
        refs.append(ex["tgt_text"])              # EN ref
        if len(buf_src) >= B:
            preds += generate_translate(buf_src)
            buf_src = []
    if buf_src:
        preds += generate_translate(buf_src)
    bleu = metric_bleu.compute(predictions=preds, references=[[r] for r in refs])
    chrf = metric_chrf.compute(predictions=preds, references=refs)
    return {"bleu": round(bleu["score"], 4), "chrf": round(chrf["score"], 4)}

if LEGACY:
    print("[EVAL] Validation (manuel)...")
    metrics_val = eval_split_manual(dsdict["validation"])
    print("Validation:", metrics_val)
    if "test" in dsdict:
        print("[EVAL] Test (manuel)...")
        metrics_test = eval_split_manual(dsdict["test"])
        print("Test:", metrics_test)
else:
    print("[EVAL] Validation (intégrée)...")
    metrics_val = trainer.evaluate(max_length=GEN_MAX_NEW_TOKENS, num_beams=BEAM_SIZE)
    metrics_val["perplexity"] = math.exp(metrics_val["eval_loss"]) if metrics_val.get("eval_loss", 99) < 20 else float("inf")
    print(metrics_val)
    if "test" in tokenized:
        print("[EVAL] Test (intégrée)...")
        test_metrics = trainer.evaluate(
            eval_dataset=tokenized["test"],
            max_length=GEN_MAX_NEW_TOKENS,
            num_beams=BEAM_SIZE,
            metric_key_prefix="test"
        )
        test_metrics["test_perplexity"] = math.exp(test_metrics["test_loss"]) if test_metrics.get("test_loss", 99) < 20 else float("inf")
        print(test_metrics)


# 8) Démo FR->EN
SAMPLES_FR = [
    "Je t'apprécie beaucoup.",
    "Où se trouve la gare la plus proche ?",
    "Ce livre a été écrit par un scientifique renommé.",
    "Pourrais-tu m'aider sur ce projet ?",
]
print("\nTraductions de démonstration (FR→EN):")
for s in SAMPLES_FR:
    out = generate_translate([PREFIX + s])[0]
    print(f"FR: {s}\nEN: {out}\n")



[TRAIN] Device: cuda
[TRAIN] Chargement du DatasetDict...
{'train': 720600, 'validation': 11143, 'test': 11144}


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

tokenize::test:   0%|          | 0/11144 [00:00<?, ? examples/s]

[TRAIN] Transformers version: 4.57.3
⚠️  Mode LEGACY activé : évaluation intégrée indisponible, on évaluera après l'entraînement.
[TRAIN] Démarrage de l'entraînement...


/tmp/ipykernel_478/2118881286.py:154: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
200,2.146200
400,1.911300
600,1.852700
800,1.787500
1000,1.741900
1200,1.723400
1400,1.732200
1600,1.698200
1800,1.679700
2000,1.660400


[EVAL] Validation (manuel)...
Validation: {'bleu': 41.6465, 'chrf': 62.2474}
[EVAL] Test (manuel)...
Test: {'bleu': 41.8978, 'chrf': 62.5291}

Traductions de démonstration (FR→EN):
FR: Je t'apprécie beaucoup.
EN: I appreciate you a lot.

FR: Où se trouve la gare la plus proche ?
EN: Where is the nearest train station?

FR: Ce livre a été écrit par un scientifique renommé.
EN: This book was written by a renamed scientist.

FR: Pourrais-tu m'aider sur ce projet ?
EN: Could you help me with this project?



### **Commentaire**
Le modèle FR→EN obtient un BLEU d’environ 41.8 et un CHRF d’environ 62.5, ce qui indique une bonne qualité de traduction globale. Sur les exemples manuels, les phrases simples sont bien traduites et quasi bien reformulées en anglais naturel.

On note toutefois une petite erreur sur renommé → renamed, ce qui montre que le modèle reste sensible à certains faux-amis, mais la structure et le sens général demeurent corrects.

# **Conclusion / limites /perspectives / acquis d'apprentissage**

### Limites

Malgré ces résultats encourageants, notre approche présente plusieurs **limites** :

- **Spécialisation des modèles**  
  Chaque modèle est entraîné dans **une seule direction**. Il est donc nécessaire de maintenir **deux modèles séparés** pour couvrir EN->FR et FR->EN, ce qui complique un peu le déploiement. Cela a été fait ainsi par manque de temps, mais un seul modèle T5 aurait pu être entraîné pour la traduction bidirectionnelle, en lui fournissant des exemples dans les deux sens (EN->FR et FR->EN) avec une indication explicite de la direction souhaitée.

- **Domaine des données (EN->FR)**  
  Le modèle EN->FR est entraîné quasi exclusivement sur **Europarl**, donc exposé surtout à un **registre politique formel**. Sa généralisation à des textes informels, techniques, ou très spécialisés reste incertaine.

- **Qualité et variété des données (FR->EN)**  
  Même si le corpus FR->EN est plus varié (Europarl + Opus Books + Francophonia), il reste constitué de phrases en général propres et bien écrites. La robustesse face à du **langage bruité** (fautes, réseaux sociaux, etc.) n’a pas été évaluée.

- **Longueur des séquences**  
  Nous avons travaillé avec des **longueurs maximales d’entrée/sortie bornées** (par exemple 128 tokens à la génération). 

- **Adaptation partielle avec LoRA**  
  LoRA réduit fortement le nombre de paramètres entraînables, ce qui est très intéressant en pratique, simplifie cette tache du fait que l'on ne risque pas de "détruire" le modèle mais peut aussi **limiter les gains "ultimes"** par rapport à un fine-tuning complet.

- **Évaluation surtout automatique**  
  Nous nous appuyons principalement sur des **métriques automatiques** (BLEU, METEOR, ROUGE, chrF). Et nous avons uniquement regarder manuellement que quelques phrases. Nous n’avons donc pas réalisé de véritable **évaluation humaine systématique**, ce qui limite la mesure de la qualité perçue par des utilisateurs finaux. **L'usage prévaut sur la règle**.

### Perspectives d’amélioration

Plusieurs **pistes** peuvent être explorées pour aller plus loin :

- **Vers un modèle réellement bidirectionnel**  
  En s’inspirant du paradigme T5 “text-to-text”, on pourrait entraîner **un seul modèle** de manière à mutualiser les capacités et simplifier le déploiement.

- **Enrichissement et diversification des corpus**  
  Ajouter des données de domaines variés (presse, dialogues, textes techniques, données bruitées) permettrait de rendre les modèles **plus robustes** et plus utilisables (meilleur) dans des situations réelles.

- **Exploration d’autres méthodes**  
  Tester d’autres approches que LoRA (prefix-tuning, adapters, etc.) pourrait permettre de trouver un meilleur compromis entre **nombre de paramètres entraînables**, **stabilité** et **performance**.

- **Affinage des hyperparamètres et du décodage**  
  Une recherche plus fine sur le **learning rate**, le **nombre d’épochs**, le **rang LoRA**, ainsi que sur les paramètres de génération (*num_beams*, *length_penalty*, *no_repeat_ngram_size*, etc.) pourrait encore améliorer les résultats.

### Acquis d’apprentissage

Ce projet nous a permis d’acquérir et de consolider plusieurs **compétences clés** :

- Mise en place d’une **pipeline de traduction de bout en bout** : chargement des données (datasets HuggingFace), prétraitement, split train/validation/test, entraînement, évaluation et tests manuels.

- Compréhension pratique du modèle **T5** et du paradigme **“text-to-text”**, avec la gestion de l’encodage, des labels, et de la valeur `-100` pour ignorer certains tokens dans la perte.

- Maîtrise de l’**adaptation par LoRA** via la librairie PEFT : configuration (TaskType *SEQ_2_SEQ_LM*, *r*, *lora_alpha*, *lora_dropout*), impact sur le nombre de paramètres entraînables et sur la consommation mémoire.

- Capacité à **gérer les ressources** (GPU/RAM) : utilisation de *gradient_accumulation_steps*, d’un callback de garbage collector, de batchs adaptés et d’un échantillonnage pour travailler sur plusieurs centaines de milliers de paires de phrases.

- Utilisation et interprétation de **métriques de traduction** (BLEU, METEOR, ROUGE, chrF), avec une compréhension de ce que chacune mesure et de leurs limites.

En résumé, nous avons construit **deux modèles de traduction spécialisés et complémentaires**, tout en explorant des techniques d’adaptation légère. Ce travail nous a donné une vision concrète des compromis entre **qualité de traduction**, **quantité et nature des données** et **contraintes de calcul**.